#**Capital bike diaries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pygeohash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00


In [3]:
import os
from pathlib import Path
import scipy
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import sklearn
import folium
import pygeohash as pgh

##Cleaning and data integration

In [4]:
!cp -r "/content/drive/MyDrive/Dataset data" /content/


In [ ]:
folder_path = Path('/content/Dataset data/')
#بيانات الرحلات اليومية
df_rent = pd.read_parquet(folder_path / "daily-rent.parquet") #df_rent: يحتوي على كافة الرحلات اليومية لاستئجار الدراجات
df_sampled_rent = pd.read_parquet(folder_path / "daily-rent-sampled.parquet") # نسخة مُصغّرة أو عيّنة من df_rent
#بيانات الطقس
df_weather = pd.read_csv(folder_path / "Washington,DC,USA 2024-01-01 to 2024-12-31.csv") # يحتوي على بيانات الطقس اليومية في واشنطن العاصمة من 1 يناير إلى 31 ديسمبر 2024.
#بيانات المحطات
df_stations_table = pd.read_csv(folder_path / "stations.csv") #جدول إضافي للمحطات، قد يحتوي على تفاصيل أخرى مثل معرّف المحطة (ID)، الاسم، عدد الأرصفة، إلخ
df_stations_info = pd.read_csv(folder_path / "Capital_Bikeshare_Locations.csv")  #معلومات حول مواقع محطات استئجار الدراجات الإحداثيات، اسم المحطة، إلخ.
#بيانات جغرافية ومناطق
df_cbd = gpd.read_file(folder_path / "DDOT_Central_Business_District.geojson") # المنطقة التجارية المركزية – تُستخدم لتحديد ما إذا كانت الرحلة بدأت أو انتهت داخل هذه المنطقة.
df_zones = gpd.read_file(folder_path / "Residential_and_Visitor_Parking_Zones.geojson") #مناطق وقوف السيارات السكنية والزوار (Residential and Visitor Parking Zones).
#بيانات النقل العام
df_metro = pd.read_csv(folder_path / "Metro_Bus_Stops.csv") #مواقف حافلات المترو (Metro Bus Stops).
df_shuttle = pd.read_csv(folder_path / "Shuttle_Bus_Stops.csv") # مواقف حافلات النقل الصغيرة (Shuttle Bus Stops).



In [11]:
df_rent.shape

(6114323, 13)

In [12]:
df_rent.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [13]:
df_rent.dtypes

,0
ride_id,string[python]
rideable_type,string[python]
started_at,datetime64[ns]
ended_at,datetime64[ns]
start_station_name,string[python]
start_station_id,string[python]
end_station_name,string[python]
end_station_id,string[python]
start_lat,float64
start_lng,float64


In [14]:
df_rent.nunique()

,0
ride_id,6114182
rideable_type,2
started_at,5866403
ended_at,5869571
start_station_name,818
start_station_id,803
end_station_name,818
end_station_id,803
start_lat,468450
start_lng,501510


In [15]:
df_rent.head(40)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,748A93D7DE8A41CD,classic_bike,2024-01-25 15:49:59,2024-01-25 15:52:35,1st & O St NW,31519,1st & L St NW,31677,38.908643,-77.012365,38.903819,-77.011987,member
1,75CBFD136F06305B,classic_bike,2024-01-02 16:44:58,2024-01-02 16:53:25,1st & O St NW,31519,4th & College St NW,31138,38.908643,-77.012365,38.921233,-77.018135,member
2,0536C9720F87E04C,classic_bike,2024-01-24 15:40:15,2024-01-24 15:43:55,1st & O St NW,31519,1st & L St NW,31677,38.908643,-77.012365,38.903819,-77.011987,member
3,9E17390C218783B5,classic_bike,2024-01-04 15:35:00,2024-01-04 15:37:35,1st & O St NW,31519,1st & L St NW,31677,38.908643,-77.012365,38.903819,-77.011987,member
4,00727D0E773CDFF7,electric_bike,2024-01-05 12:27:58,2024-01-05 12:35:40,1st & O St NW,31519,10th & G St NW,31274,38.908690,-77.012317,38.898243,-77.026235,casual
5,CFE1D1300DA4B471,classic_bike,2024-01-02 08:04:00,2024-01-02 08:20:06,1st & O St NW,31519,Washington & Independence Ave SW/HHS,31272,38.908643,-77.012365,38.886978,-77.013769,casual
6,9645567A2A28DD0C,classic_bike,2024-01-03 08:30:20,2024-01-03 08:43:36,1st & O St NW,31519,Washington & Independence Ave SW/HHS,31272,38.908643,-77.012365,38.886978,-77.013769,casual
7,33A3DDDBA995093F,electric_bike,2024-01-28 16:03:16,2024-01-28 16:10:47,11th & Park Rd NW,31651,15th & W St NW,31125,38.931437,-77.028228,38.919018,-77.034449,member
8,EC4E369AEB78721E,electric_bike,2024-01-28 12:54:01,2024-01-28 13:00:03,11th & Park Rd NW,31651,15th & W St NW,31125,38.931390,-77.028129,38.919018,-77.034449,member
9,80E3C3187C373418,electric_bike,2024-01-27 14:15:15,2024-01-27 14:27:40,Wilson Blvd & N Oak St,31939,<NA>,<NA>,38.895017,-77.073763,38.910000,-77.060000,member


In [16]:
print(df_rent.duplicated().sum())

0


In [17]:
df_rent.isnull().sum()

,0
ride_id,0
rideable_type,0
started_at,0
ended_at,0
start_station_name,1190966
start_station_id,1190966
end_station_name,1236167
end_station_id,1237694
start_lat,0
start_lng,0


In [13]:
print((df_rent['start_station_id'] == df_rent['end_station_id']).mean())

0.0441441028702274


In [37]:
multi_names = (
    df_rent.dropna(subset=['end_station_id', 'end_station_name'])
    .groupby('end_station_id')['end_station_name']
    .nunique()
)

print(multi_names[multi_names > 1])


end_station_id
31045    2
31133    3
31225    2
31339    2
31380    2
31401    2
31407    2
31502    2
31528    2
31613    2
31660    2
31699    2
31726    2
31949    2
32288    2
Name: end_station_name, dtype: int64


In [36]:
only_name_missing = df_rent['start_station_name'].isna() & df_rent['start_station_id'].notna()
only_id_missing   = df_rent['start_station_id'].isna() & df_rent['start_station_name'].notna()
both_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna()
none_missing      = df_rent['start_station_id'].notna() & df_rent['start_station_name'].notna()
both4_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna() & df_rent['end_station_id'].isna() & df_rent['end_station_name'].isna()

both23_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna() & df_rent['end_station_id'].notna() & df_rent['end_station_name'].isna()
both32_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna() & df_rent['end_station_id'].isna() & df_rent['end_station_name'].notna()
both2_missing      = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna() & df_rent['end_station_id'].notna() & df_rent['end_station_name'].notna()
eonly_name_missing = df_rent['end_station_name'].isna() & df_rent['end_station_id'].notna()
eonly_id_missing   = df_rent['end_station_id'].isna() & df_rent['end_station_name'].notna()
eboth_missing  = df_rent['end_station_id'].isna() & df_rent['end_station_name'].isna()
print("✅ عدد الصفوف التي فقط الاسم مفقود:", only_name_missing.sum())
print("✅ عدد الصفوف التي فقط المعرف مفقود:", only_id_missing.sum())
print("✅ عدد الصفوف التي كلاهما مفقود:", both_missing.sum())
print("✅ عدد الصفوف التي لا يوجد بها فقد:", none_missing.sum())

print("✅td  عدد الصفوف التي  يوجد بها فقد:", both4_missing.sum())

print("✅td  عدد الصفوف التي  يوجد بها فقد:", both23_missing.sum())
print("✅td  عدد الصفوف التي  يوجد بها فقد:", both32_missing.sum())
print("✅td  عدد الصفوف التي  يوجد بها فقد:", both2_missing.sum())
print("✅ عدد الصفوف التي فقط الاسم مفقود:", eonly_name_missing.sum())
print("✅ عدد الصفوف التي فقط المعرف مفقود:", eonly_id_missing.sum())
print("✅ عدد الصفوف التي كلاهما مفقود:", eboth_missing.sum())

✅ عدد الصفوف التي فقط الاسم مفقود: 0
✅ عدد الصفوف التي فقط المعرف مفقود: 0
✅ عدد الصفوف التي كلاهما مفقود: 0
✅ عدد الصفوف التي لا يوجد بها فقد: 6114323
✅td  عدد الصفوف التي  يوجد بها فقد: 0
✅td  عدد الصفوف التي  يوجد بها فقد: 0
✅td  عدد الصفوف التي  يوجد بها فقد: 0
✅td  عدد الصفوف التي  يوجد بها فقد: 0
✅ عدد الصفوف التي فقط الاسم مفقود: 0
✅ عدد الصفوف التي فقط المعرف مفقود: 0
✅ عدد الصفوف التي كلاهما مفقود: 0


In [25]:
# تعبئة صفوف محطة البداية فقط عند فقدان كلي (اسم ومعرّف)
most_common_start = df_rent[['start_station_name', 'start_station_id']].mode().iloc[0]

mask_both_start_missing = df_rent['start_station_id'].isna() & df_rent['start_station_name'].isna()

df_rent.loc[mask_both_start_missing, 'start_station_id'] = most_common_start['start_station_id']
df_rent.loc[mask_both_start_missing, 'start_station_name'] = most_common_start['start_station_name']


In [30]:
# بناء قاموس من الاسم إلى المعرف (الأكثر شيوعًا)
end_name_to_id = (
    df_rent.dropna(subset=['end_station_id', 'end_station_name'])
    .groupby('end_station_name')['end_station_id']
    .agg(lambda x: x.mode().iloc[0])
)

# قناع للصفوف التي فيها الاسم موجود والمعرف مفقود
mask_eonly_id_missing = df_rent['end_station_id'].isna() & df_rent['end_station_name'].notna()

# تعبئة end_station_id
df_rent.loc[mask_eonly_id_missing, 'end_station_id'] = df_rent.loc[mask_eonly_id_missing, 'end_station_name'].map(end_name_to_id)


In [34]:
# الأكثر تكرارًا لكل start_station_id
most_common_end_per_start = (
    df_rent.dropna(subset=['start_station_id', 'end_station_id'])
    .groupby('start_station_id')['end_station_id']
    .agg(lambda x: x.mode().iloc[0])
)

# قناع للصفوف التي نهاية المحطة مفقودة كليًا
mask_missing_end = df_rent['end_station_id'].isna() & df_rent['end_station_name'].isna() & df_rent['start_station_id'].notna()

# تعبئة end_station_id بالتخمين من البداية
df_rent.loc[mask_missing_end, 'end_station_id'] = df_rent.loc[mask_missing_end, 'start_station_id'].map(most_common_end_per_start)
# بناء قاموس end_id إلى name
end_id_to_name = (
    df_rent.dropna(subset=['end_station_id', 'end_station_name'])
    .groupby('end_station_id')['end_station_name']
    .agg(lambda x: x.mode().iloc[0])
)

# تعبئة الاسم بعد تعبئة المعرف
df_rent.loc[mask_missing_end, 'end_station_name'] = df_rent.loc[mask_missing_end, 'end_station_id'].map(end_id_to_name)


In [35]:
print((df_rent['start_station_id'] == df_rent['end_station_id']).mean())

0.04967778117054006


##Exploration and Analysis

##Pattern discovery

##Documentation